<a href="https://colab.research.google.com/github/angelblancas/03MAIR-Algoritmos-de-Optimizacion/blob/master/AG2/AG2_Angel_Blancas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Ángel Blancas García - AG2<br>
 Actividad Guiada<br>
 URL: [https://github.com/angelblancas/03MAIR-Algoritmos-de-Optimizacion/tree/master/AG2](https://github.com/angelblancas/03MAIR-Algoritmos-de-Optimizacion/tree/master/AG2)

**ALGORITMO MENOR DISTANCIA ENTRE DOS PUNTOS - MEJORAS REALIZADAS:**

1.   Resuelvo la zona de los puntos que quedan a uno y otro lado del pivote dentro de la función recursiva teniendo en cuenta posibles repeticiones del punto pivote en las listas
2.   Control de varios errores, por ejemplo si llega un solo elemento en la lista, si llegan elementos de diferente longitud, se deja batería de pruebas en la definición de la lista 2D
3.   Función de ordenación de la lista fuera del bucle de recursividad para mejorar tiempos
4.   Antes de inciar la recursividad compruebo si hay puntos repetidos en la lista, si los hay la menor distancia es 0 con esos puntos
5.   Devolución además de los dos puntos de la distancia.



In [0]:
#Decorador para calculo del tiempo de una función modificado para mejor salida de información
from time import time
def calcular_tiempo(f):
  def wrapped(*args, **kw):
    ti = time()
    result = f(*args, **kw)
    te = time() - ti
    print ('\nTiempo de ejecución de la funcion %r: %2.5f ms'%(f.__name__,te*1000))
    return result
  return wrapped

In [0]:
# Importación de librerías necesarias
import random
import math
import numpy as np

In [0]:
# Definición de lista 2D
N= 1000 # Número de elementos para el modelo aleatorio
LISTA_2D= [(random.randrange(1,N*10), random.randrange(1,N*10))for _ in range(N)] # Modelo aleatorio
#LISTA_2D=[(0,5)] # Control de error para cuando llega un solo elemento en la lista
#LISTA_2D=[(0,5),(2,)]
#LISTA_2D=[(0,0),(10,0),(20,0),(25,0),(89,0),(45,0),(46,0),(48,0),(66,0),(70,0),(80,0),(25,0),(80,0)] # Modelo no aleatorio para probar cuando hay puntos repetidos
#LISTA_2D=[(0,0),(2,0),(20,0),(25,0),(35,0),(45,0),(46,0),(48,0),(66,0),(70,0),(80,0),(90,0)] # Modelo no aleatorio para probar la zona cercana a los pivotes 

#print (LISTA_2D)
#print ('Longitud de la lista: ',len(LISTA_2D))

In [0]:
# Función Distancia entre dos elementos A y B
def distancia (A,B):
  if type(A) is int or type(A) is float: # Si estamos ante un elemento entero o flotante
    return abs(B-A)
  else:                                  # Si estamos ante elementos n dimensionales
    if(len(A)==len(B)):
      return math.sqrt(sum([(A[i]-B[i])**2 for i in range(len(A))]))
    else:
      print('# ERROR (distancia): Los parámetros deben tener la misma longitud') 
      return -1 
      

In [0]:
# Algoritmo FUERZA BRUTA
def distancia_fuerza_bruta(L):
                              # Como vamos a medir múltiples veces la longitud lo almaceno en una variable
  numero_Puntos = len(L)
  if numero_Puntos<2:         # Control de errores por si llegan 1 o ningún punto.
    print('# ERROR (distancia_fuerza_bruta): Esto es un problema, no deberías pedirme calcular distancia si no hay mínimo dos puntos')
    return  
  mejor_distancia = 100000e10 # Inicializamos con un valor muy alto para asegurarnos cumplimiento
  A,B=(),()                   # Almacenamos los Mejores valores encontrados hasta el momento
  for i in range(numero_Puntos):
    for j in range(i+1,numero_Puntos):
      d= distancia(L[i],L[j])
      if  d < mejor_distancia:
        A,B = L[i], L[j]
        mejor_distancia = d
  return (mejor_distancia,[A,B]) 
  

In [0]:
# Debido a que esta acción se realizará una sola vez lo separamos en función independiente evitando que esté dentro del bucle recursivo
def ordena_Lista(L):
  L_ORDENADA= sorted(L,key=lambda x: x[0])           # Ordeno la lista
  longLista=len(L_ORDENADA)    
  for i in range(len(L_ORDENADA)):                   # Recorremos la lista en busca de puntos repetidos
    if i<longLista-1:
       if L_ORDENADA[i]==L_ORDENADA[i+1]:
        return (0,[L_ORDENADA[i],L_ORDENADA[i+1]])   # Si encuentro dos puntos repetidos devuelvo distancia, punto A y punto B evitando entrar en bucle recursivo
  return distancia_divide_venceras_AB(L_ORDENADA)    # Si no hay elementos repetidos llamo al algoritmo divide y venceras al que ya paso la lista ordenada

In [0]:
# Algorimo DIVIDE Y VENCERAS
def distancia_divide_venceras_AB(L):
  numero_Puntos = len(L)
  if numero_Puntos <= 3:
    return distancia_fuerza_bruta(L)
 
  # Dividimos la lista  
  LISTA_IZQ = L[:numero_Puntos//2]
  LISTA_DER = L[numero_Puntos//2:]
  PUNTO_PIVOTE = LISTA_IZQ[-1]
  dIZQ, PUNTOS_LISTA_IZQ = distancia_divide_venceras_AB(LISTA_IZQ)
  dDER, PUNTOS_LISTA_DER = distancia_divide_venceras_AB(LISTA_DER)
  dMIN, PUNTOS_MIN= (dIZQ, PUNTOS_LISTA_IZQ) if dIZQ < dDER else (dDER, PUNTOS_LISTA_DER)    #Control de la distancia mínima para resolver los áreas cercanas al pivote

  # Genero una lista de puntos dentro de dMIN respecto al pivote
  cercanosdMIN =[punto for punto in L if distancia(punto,PUNTO_PIVOTE) <dMIN ]
  nCercanosdMIN = len(cercanosdMIN)
  if nCercanosdMIN > 1:
    PUNTOS_INTERMEDIOS = cercanosdMIN
  else:
    PUNTOS_INTERMEDIOS = []
 
  # Hay un caso de error cuando el punto pivote está en la lista de la derecha o izquierda además de la de puntos intermedios como sabemos que no había repetidos el la lista se eliminan
  _PUNTOS_FINAL = PUNTOS_LISTA_IZQ+PUNTOS_INTERMEDIOS+PUNTOS_LISTA_DER
  PUNTOS_FINAL =list(set(_PUNTOS_FINAL))

  return distancia_fuerza_bruta (PUNTOS_FINAL)

In [0]:
@calcular_tiempo
def dist_div_venceras(L):
  #Para Evitar entrar en recursivo para funciones como ordenar la lista o comprobar si hay dos repetidos los saco a dos funciones
  
  return ordena_Lista(L)

In [0]:
dist_div_venceras(LISTA_2D)


Tiempo de ejecución de la funcion 'dist_div_venceras': 36.53836 ms


(7.0710678118654755, [(3229, 423), (3222, 424)])

**ALGORITMO VIAJE POR EL RÍO - MEJORAS REALIZADAS**


1.   El algoritmo original falla en punto intermedios, por ejemplo ir de 3 a 6
2.   El algoritmo no permirtía calcular caminos posibles que existieran desde un nodo posterior a uno anterior. 
3.   Se ha corregido también la salida por pantalla indicando de forma correcta el camino, en el original se repetía el origen y no salía impreso el destino. También se puede ver ahora el peso del camino.
4.   Se ha corregido errores cuando se pide rutas no existentes, como la origen 1 destino 2, el algorimo original devolvía 1,2. En realidad esa ruta no existe, lo he modificado teniendo presente los pesos que hemos puesto como límite

In [0]:
TARIFAS = [
    [0,5,4,3,999,999,999],           # Indica las tarifas de viajar desde el nodo 0 al resto de nodos lógicamente del 0 al 0 tiene una TARIFA de 0 y los nodos a los que no llego en directo tienen 999
    [999,0,999,2,3,999,11],          # Indica las taridas de viajar desde el nodo 1 al resto de nodos 
    [999,999,0,1,999,4,10],          # Indica las taridas de viajar desde el nodo 2 al resto de nodos 
    [999,999,999,0,5,6,9],           # Indica las taridas de viajar desde el nodo 3 al resto de nodos 
    [999,999,999,999,0,999,4],       # Indica las taridas de viajar desde el nodo 4 al resto de nodos 
    [999,999,999,999,999,0,3],       # Indica las taridas de viajar desde el nodo 5 al resto de nodos 
    [999,999,999,999,999,999,0]      # Indica las taridas de viajar desde el nodo 6 al resto de nodos, como no hay vueltra atrá la tarifa es siempre 999 salvo cuando viaja de uno a si mismo
]

In [0]:
# VIAJE POR EL RIO
def precios(TARIFAS):
  N=len(TARIFAS[0])
 
  PRECIOS = [[9999]*N for i in [9999]*N]      # Definición de matriz cuadrada de NxN con todos los elementos inicialidos a 9999, es mayor que cualquier valor de la tabla TARIFAS
  RUTA =[[""]*N for i in [""]*N]
  
 
  for i in range(N-1):
    for j in range (N):
      MIN = TARIFAS[i][j]
      RUTA[i][j]=i    
      for k in range(i,N):
        if k != j: 
          if PRECIOS[i][k] == 9999:     # Implica que este camino aún no está evaluado
            if TARIFAS[i][k]+TARIFAS[k][j] < MIN:
              MIN= min(MIN,TARIFAS[i][k]+TARIFAS[k][j])
              RUTA[i][j] = k
          elif PRECIOS[i][k]+TARIFAS[k][j] < MIN:
            MIN= min(MIN,PRECIOS[i][k]+TARIFAS[k][j])
            RUTA[i][j] = k
        PRECIOS[i][j]=MIN
  return PRECIOS, RUTA


In [0]:
def calcular_ruta(PRECIO, RUTA, desde, hasta):
  if desde == hasta:
    return str(desde)      
  else:
    if PRECIO[desde][hasta]>=999:
      return str('ESTA RUTA NO EXISTE')
    return calcular_ruta(PRECIO, RUTA,desde,RUTA[desde][hasta]) + ','+ str(hasta)
  

In [115]:
PRECIO, RUTA= precios(TARIFAS)
d=0
h=6

print('\nLa ruta desde',d,"hasta",h,'pasa por:')
print(calcular_ruta(PRECIO, RUTA, d,h))
if PRECIO[d][h]< 999:
  print('Y tiene un coste de: ',PRECIO[d][h])



La ruta desde 0 hasta 6 pasa por:
0,2,5,6
Y tiene un coste de:  11
